In [ ]:
# Resources: Tutorial by Nicholas Renotte (https://www.youtube.com/watch?v=We1uB79Ci-w&ab_channel=NicholasRenotte)

In [ ]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv
import os
import numpy as np #array processing

In [ ]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # have t convert colors
        image.flags.writeable = False        #()
        
        # Make Detections
        results = holistic.process(image)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
num_coords = len(results.pose_landmarks.landmark)+len(results.face_landmarks.landmark)  #checking how many landmarks we have
num_coords

In [ ]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
class_name = "Namaste"

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # have t convert colors
        image.flags.writeable = False        #()
        
        # Make Detections
        results = holistic.process(image)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concat rows
            row = pose_row+face_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV (mode=a to append)
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [16]:
class_names = ['Tree Pose', 'Warrior Pose', 'Namaste']

In [78]:
df = pd.read_csv('coords.csv')

In [79]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Tree Pose,0.504519,0.086591,-0.769682,0.997681,0.538304,-0.001652,-0.779766,0.993583,0.554626,...,-0.007057,0.0,0.587464,-0.069674,0.009867,0.0,0.593077,-0.081704,0.010568,0.0
1,Tree Pose,0.478097,0.037287,-0.917488,0.996928,0.500994,-0.031469,-0.889720,0.992403,0.516402,...,-0.001274,0.0,0.537659,-0.001347,0.009887,0.0,0.542336,-0.010833,0.010421,0.0
2,Tree Pose,0.532065,0.084483,-0.347586,0.998314,0.542497,0.043032,-0.330007,0.995890,0.550547,...,0.000345,0.0,0.566211,0.053980,0.004027,0.0,0.569258,0.049431,0.004065,0.0
3,Tree Pose,0.571405,0.088993,-0.245444,0.998607,0.581797,0.053662,-0.226708,0.996634,0.588699,...,0.003836,0.0,0.596013,0.081241,0.010257,0.0,0.598324,0.078645,0.010485,0.0
4,Tree Pose,0.578885,0.095348,-0.198969,0.998716,0.589672,0.062347,-0.183288,0.996933,0.597069,...,0.002270,0.0,0.606339,0.082727,0.007392,0.0,0.608620,0.079475,0.007545,0.0


In [80]:
df.loc[df['class']=='Tree Pose', 'class'] = 0
df.loc[df['class']=='Warrior Pose', 'class'] = 1
df.loc[df['class']=='Namaste', 'class'] = 2
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0,0.504519,0.086591,-0.769682,0.997681,0.538304,-0.001652,-0.779766,0.993583,0.554626,...,-0.007057,0.0,0.587464,-0.069674,0.009867,0.0,0.593077,-0.081704,0.010568,0.0
1,0,0.478097,0.037287,-0.917488,0.996928,0.500994,-0.031469,-0.889720,0.992403,0.516402,...,-0.001274,0.0,0.537659,-0.001347,0.009887,0.0,0.542336,-0.010833,0.010421,0.0
2,0,0.532065,0.084483,-0.347586,0.998314,0.542497,0.043032,-0.330007,0.995890,0.550547,...,0.000345,0.0,0.566211,0.053980,0.004027,0.0,0.569258,0.049431,0.004065,0.0
3,0,0.571405,0.088993,-0.245444,0.998607,0.581797,0.053662,-0.226708,0.996634,0.588699,...,0.003836,0.0,0.596013,0.081241,0.010257,0.0,0.598324,0.078645,0.010485,0.0
4,0,0.578885,0.095348,-0.198969,0.998716,0.589672,0.062347,-0.183288,0.996933,0.597069,...,0.002270,0.0,0.606339,0.082727,0.007392,0.0,0.608620,0.079475,0.007545,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,2,0.465092,0.175009,-0.342864,0.999588,0.478066,0.141664,-0.309630,0.999218,0.487323,...,0.002417,0.0,0.489378,0.129727,0.009471,0.0,0.491741,0.125295,0.009745,0.0
463,2,0.465455,0.165293,-0.355368,0.999627,0.478515,0.128292,-0.323451,0.999292,0.487900,...,0.000708,0.0,0.493797,0.116602,0.009041,0.0,0.496198,0.111310,0.009364,0.0
464,2,0.467575,0.152983,-0.364810,0.999662,0.481729,0.113485,-0.332944,0.999354,0.492069,...,0.002074,0.0,0.498761,0.099536,0.009823,0.0,0.501687,0.093946,0.010178,0.0
465,2,0.473853,0.131834,-0.469234,0.999672,0.492597,0.089371,-0.438844,0.999375,0.503041,...,0.001699,0.0,0.508868,0.074014,0.009455,0.0,0.511904,0.068228,0.009704,0.0


In [81]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,0,0.504519,0.086591,-0.769682,0.997681,0.538304,-0.001652,-0.779766,0.993583,0.554626,...,-0.007057,0.0,0.587464,-0.069674,0.009867,0.0,0.593077,-0.081704,0.010568,0.0
1,0,0.478097,0.037287,-0.917488,0.996928,0.500994,-0.031469,-0.889720,0.992403,0.516402,...,-0.001274,0.0,0.537659,-0.001347,0.009887,0.0,0.542336,-0.010833,0.010421,0.0
2,0,0.532065,0.084483,-0.347586,0.998314,0.542497,0.043032,-0.330007,0.995890,0.550547,...,0.000345,0.0,0.566211,0.053980,0.004027,0.0,0.569258,0.049431,0.004065,0.0
3,0,0.571405,0.088993,-0.245444,0.998607,0.581797,0.053662,-0.226708,0.996634,0.588699,...,0.003836,0.0,0.596013,0.081241,0.010257,0.0,0.598324,0.078645,0.010485,0.0
4,0,0.578885,0.095348,-0.198969,0.998716,0.589672,0.062347,-0.183288,0.996933,0.597069,...,0.002270,0.0,0.606339,0.082727,0.007392,0.0,0.608620,0.079475,0.007545,0.0


In [82]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [83]:
X = np.asarray(X).astype('float32')
y = np.asarray(y).astype('float32')

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234) #testing partition of 30%

In [85]:
from tensorflow import keras
from tensorflow.keras import layers

In [86]:
model = tf.keras.Sequential([layers.Dense(64),layers.Dense(1)])

In [87]:
model.compile(loss=tf.keras.losses.MeanSquaredError(),optimizer=tf.optimizers.Adam(), metrics=["accuracy"])

In [88]:
model.fit(X_train,y_train, batch_size=32, epochs=5, verbose=2)

Epoch 1/5
11/11 - 0s - loss: 2.5952 - accuracy: 0.3681 - 318ms/epoch - 29ms/step
Epoch 2/5
11/11 - 0s - loss: 0.8309 - accuracy: 0.4018 - 13ms/epoch - 1ms/step
Epoch 3/5
11/11 - 0s - loss: 0.3307 - accuracy: 0.5828 - 17ms/epoch - 2ms/step
Epoch 4/5
11/11 - 0s - loss: 0.2595 - accuracy: 0.5859 - 20ms/epoch - 2ms/step
Epoch 5/5
11/11 - 0s - loss: 0.1939 - accuracy: 0.6411 - 20ms/epoch - 2ms/step


In [89]:
model.evaluate(X_test, y_test, batch_size=32, verbose=2)

5/5 - 0s - loss: 0.1281 - accuracy: 0.6667 - 87ms/epoch - 17ms/step


[0.12814006209373474, 0.6666666865348816]

In [92]:
import tf2onnx
tf2onnx.convert.from_keras(model, output_path='model.onnx')

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


2022-08-04 12:47:52.901629: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2022-08-04 12:47:52.923091: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)


(ir_version: 7
 producer_name: "tf2onnx"
 producer_version: "1.12.0 a58786"
 graph {
   node {
     input: "dense_13_input"
     input: "sequential_8/dense_13/MatMul/ReadVariableOp:0"
     output: "sequential_8/dense_13/MatMul:0"
     name: "sequential_8/dense_13/MatMul"
     op_type: "MatMul"
   }
   node {
     input: "sequential_8/dense_13/MatMul:0"
     input: "sequential_8/dense_13/BiasAdd/ReadVariableOp:0"
     output: "sequential_8/dense_13/BiasAdd:0"
     name: "sequential_8/dense_13/BiasAdd"
     op_type: "Add"
   }
   node {
     input: "sequential_8/dense_13/BiasAdd:0"
     input: "sequential_8/dense_14/MatMul/ReadVariableOp:0"
     output: "sequential_8/dense_14/MatMul:0"
     name: "sequential_8/dense_14/MatMul"
     op_type: "MatMul"
   }
   node {
     input: "sequential_8/dense_14/MatMul:0"
     input: "sequential_8/dense_14/BiasAdd/ReadVariableOp:0"
     output: "dense_14"
     name: "sequential_8/dense_14/BiasAdd"
     op_type: "Add"
   }
   name: "tf2onnx"
   initial

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: dense_13_input for the following indices
 index: 1 Got: 1 Expected: 2004
 Please fix either the inputs or the model.

In [ ]:
# training models
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [ ]:
fit_models

In [ ]:
fit_models['rf'].predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score # Accuracy metrics for testing
#import pickle # save models down to disk

In [ ]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:
fit_models['rc'].predict(X_test)

In [ ]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

In [1]:
import pickle
# making detections
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [4]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [7]:
from sklearn.linear_model import LogisticRegression
clr = LogisticRegression()
clr.fit(X_train, y_train)
predictions = clr.predict(X_test)
score = clr.score(X_test, y_test)
print(score)

1.0


/Users/anjalisingh/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("logreg_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # have t convert colors
        image.flags.writeable = False        #()
        
        # Make Detections
        results = holistic.process(image)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concat rows
            row = pose_row+face_row
            
            # Append class name 
            #row.insert(0, class_name)
            
            # Export to CSV (mode=a to append)
            #with open('coords.csv', mode='a', newline='') as f:
                #csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                #csv_writer.writerow(row) 

            # Make detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)

            # PRINTING/RENDERING INFO ON WEBCAM: taken straight from tutorial (will update with game engine)

            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

RuntimeError: Input must be a list of dictionaries or a single numpy array for input 'float_input'.

RuntimeError: Input must be a list of dictionaries or a single numpy array for input 'float_input'.